In [56]:
# still too-good-to-be-true(aka wrong) values, will fix; correct upto score calculation at least
import math
import numpy as np
import torch
from data_loader import get_dataset

# X_test, y_test = read_data('test.txt')
with open('model.txt') as f:
    content=f.readlines()
x=[float(x.rstrip()) for x in content]


num_features = 128 #128
num_label = int((math.sqrt(num_features**2+4*len(x)) - num_features)/2) #26
W = np.reshape(x[:num_features*num_label], (num_features, num_label), order='F')
T = np.reshape(x[num_features*num_label:], (num_label, num_label), order='F')


In [ ]:
dataset = get_dataset()
split = int(0.5 * len(dataset.data)) # train-test split
train_data, test_data = dataset.data[:split], dataset.data[split:]
train_target, test_target = dataset.target[:split], dataset.target[split:]

In [43]:
def Messages(x, word_list, word_labels):
    num_ex=len(word_list)
    w=word_list[0]
    num_features = len(w[1]) #128
    num_label = int((math.sqrt(num_features**2+4*len(x)) - num_features)/2) #26
    W = np.reshape(x[:num_features*num_label], (num_features, num_label), order='F')
    T = np.reshape(x[num_features*num_label:], (num_label, num_label), order='F')
    l = np.zeros((num_ex,num_label, 100))
    r = np.zeros((num_ex,num_label, 100))

    for ex in range(num_ex):
#         word = word_list[ex]
#         word_label = word_labels[ex]
#         num_letter = len(word)
        word_label = np.nonzero(word_labels[ex])[1]
        word = word_list[ex,:len(word_label)]      
        num_letter = len(word_label)
        

        def letter_func(l):
#             print(W)
#             print(np.matmul(l,W))
            return np.matmul(l,W)
        
        score = np.transpose(np.apply_along_axis(letter_func, 1, word))
        

        l[ex,:,0] = 0
        r[ex,:,num_letter-1] = 0
        for i in range(1, num_letter):
            v = np.reshape(np.add(l[ex,:,i-1], score[:,i-1]), (num_label,1))
#             print(v.shape)
            temp = np.add(T, np.tile(v, (1,num_label)))
#             print(temp.shape)
            max_temp = np.reshape(np.amax(temp, axis=0), (1,num_label))
#             print(max_temp.shape)
            l[ex,:, i] = np.add(np.log(np.sum(np.exp(np.subtract(temp, np.tile(max_temp, (num_label,1)))), axis=0)), max_temp)
            
        for i in range(num_letter-2, -1, -1):
            v = np.reshape(np.add(r[ex,:,i+1], score[:,i+1]), (num_label,1))  
            temp = np.add(T, np.tile(np.transpose(v), (num_label,1)))
            max_temp = np.reshape(np.amax(temp, axis=1), (num_label,1))
#             print(max_temp.shape)
            r[ex,:, i] = np.transpose(np.add(np.log(np.reshape(np.sum(np.exp(np.subtract(temp, np.tile(max_temp, (1,num_label)))), axis=1), (26,1))), max_temp))
#             print(r.shape)
    return l,r

In [49]:
def compute_log_probability(word_list,word_labels,l,r,T,W):
    num_ex=len(word_labels)
    f = 0
    for ex in range(num_ex):
#         word = word_list[ex]
#         word_label = word_labels[ex]
#         num_letter = len(word)
        word_label = np.nonzero(word_labels[ex])[1]
        word = word_list[ex,:len(word_label)]      
        num_letter = len(word_label)


        def letter_func(l):
#             print(l.shape,W.shape)
            return np.matmul(l,W)
        score = np.transpose(np.apply_along_axis(letter_func, 1, word))
        
        
        l_plus_score = np.add(l[ex,:, :num_letter], score)
        r_plus_score = np.add(r[ex,:, :num_letter], score)     
        marg = np.add(l_plus_score, r_plus_score)
        marg = np.subtract(marg, score)
        t = np.amax(marg[:,0])
        f = f - math.log(np.sum(np.exp(marg[:,0]-t))) - t
        for i in range(num_letter):
                lab = word_label[i]
                f = f + score[lab][i]
                if i < num_letter-1:
                    next_lab = word_label[i+1]
                    f = f + T[lab][next_lab]
    return f

In [50]:
def ComputeGrad(word_list,word_labels,T,l,r):
    num_ex=len(word_labels)
    g_W = np.zeros(W.shape)
    g_T = np.zeros(T.shape)
    for ex in range(num_ex):
#         word = word_list[ex]
#         word_label = word_labels[ex]
#         num_letter = len(word)
        word_label = np.nonzero(word_labels[ex])[1]
        word = word_list[ex,:len(word_label)]      
        num_letter = len(word_label)
        
        def letter_func(l):
#             print(np.matmul(l,W))
            return np.matmul(l,W)
        score = np.transpose(np.apply_along_axis(letter_func, 1, word))
        
        l_plus_score = np.add(l[ex,:, :num_letter], score)
        r_plus_score = np.add(r[ex,:, :num_letter], score)     
        marg = np.add(l_plus_score, r_plus_score)
        marg = np.subtract(marg, score)
        marg = np.exp(np.subtract(marg, np.tile(np.amax(marg, axis=0), (num_label, 1))))
        marg = np.divide(marg, np.tile(np.sum(marg, axis=0), (num_label, 1)))  
        
#         print(marg.shape)
        for i in range(num_letter):
            lab = word_label[i]
            V = np.reshape(marg[:,i], (num_label,1))
            V[lab] = V[lab] - 1
            
#             print(V.shape)
            g_W = np.subtract(g_W, np.matmul(np.reshape(word[i], (num_features,1)), np.transpose(V)))
            if i < num_letter-1:
                next_lab = word_label[i+1]
                V = np.add(T, np.tile(np.reshape(l_plus_score[:,i], (num_label,1)), (1, num_label)))
                V = np.add(V, np.tile(np.transpose(r_plus_score[:,i+1]), (num_label, 1)))
                V = np.exp(V - np.amax(V))
                g_T = g_T - V / np.sum(V)
                g_T[lab][next_lab] = g_T[lab][next_lab] + 1
    grad=np.concatenate([g_W.reshape(-1), g_T.reshape(-1)])
    return grad

In [51]:
import time
s=time.time()
f,b=Messages(x,train_data, train_target)
logp=compute_log_probability(train_data, train_target,f,b,T,W)
g=ComputeGrad(train_data, train_target,T,f,b)
print(logp)
print(time.time()-s)


-102439.69961427595
4.362769603729248


In [57]:
def loss(logprob,x,C,N):
    if C>0:
        obj = np.linalg.norm(x)**2/2 - logprob*C/N
    else: 
        print(logprob,N)
        obj = logprob / N
    return obj

In [53]:
loss(logp,x,-1,len(train_data))

-102439.69961427595 3438


-29.796305879661414